In [1]:
import sys
sys.path.append("../src/")
from loss_function import loss_function
from loss_function import cs
from torch.autograd import Variable
from evaluate import Evaluation
from cnn import CNN
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sklearn
import random
import time

In [2]:
import data_reader as dr

In [3]:
corpus_path = "../data/text_tokenized.txt.gz"
corpus = dr.read_corpus(corpus_path)

In [4]:
embedding_path = "../data/vectors_pruned.200.txt.gz"
embedding_tensor, word_to_indx = dr.getEmbeddingTensor(embedding_path)

In [5]:
ids_corpus = dr.map_corpus(corpus, word_to_indx)

In [6]:
train_path = "../data/train_random.txt"
train = dr.read_annotations(train_path)

In [7]:
train_ex = dr.create_train_set(ids_corpus, train)

In [22]:
def normalize(x, dim):
    l2 = torch.norm(x, 2, dim).expand_as(x)
    return x / l2.clamp(min = 1e-8)

In [27]:
import torch.utils.data

def train_model(train_data, dev_data, model):
    model.cuda()
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)
    model.train()

    lasttime = time.time()
    for epoch in range(1, 11):
        print("-------------\nEpoch {}:\n".format(epoch))

        loss = run_epoch(train_data, True, model, optimizer)
        #return loss
        print('Train loss: {:.6f}'.format(loss))
        torch.save(model, "model{}".format(epoch))
        
        (MAP, MRR, P1, P5) = run_epoch(dev_data, False, model, optimizer)
        print('Val MAP: {:.6f}, MRR: {:.6f}, P1: {:.6f}, P5: {:.6f}'.format(MAP, MRR, P1, P5))
        
        print('This epoch took: {:.6f}'.format(time.time() - lasttime))
        lasttime = time.time()

        
def run_epoch(data, is_training, model, optimizer):
    '''
    Train model for one pass of train data, and return loss, acccuracy
    '''
    data_loader = torch.utils.data.DataLoader(
        data,
        batch_size=40,
        shuffle=True,
        num_workers=4,
        drop_last=False)

    losses = []

    if is_training:
        model.train()
    else:
        model.eval()

    for batch in data_loader:
        pid_title = torch.unsqueeze(Variable(batch['pid_title']), 1)
        pid_body = torch.unsqueeze(Variable(batch['pid_body']), 1)
        rest_title = Variable(batch['rest_title'])
        rest_body = Variable(batch['rest_body'])
        
        pid_title_pad = torch.unsqueeze(Variable(batch['pid_title_pad']), 1)
        pid_body_pad = torch.unsqueeze(Variable(batch['pid_body_pad']), 1)
        rest_title_pad = Variable(batch['rest_title_pad'])
        rest_body_pad = Variable(batch['rest_body_pad'])
        
        pid_title, pid_body = pid_title.cuda(), pid_body.cuda()
        rest_title, rest_body = rest_title.cuda(), rest_body.cuda()
        pid_title_pad, pid_body_pad = pid_title_pad.cuda(), pid_body_pad.cuda()
        rest_title_pad, rest_body_pad = rest_title_pad.cuda(), rest_body_pad.cuda()
        
        if is_training:
            optimizer.zero_grad()
        
        pt = model(pid_title)
        pb = model(pid_body)
        rt = model(rest_title)
        rb = model(rest_body)

        pt = normalize(pt, 2)
        pb = normalize(pb, 2)
        rt = normalize(rt, 2)
        rb = normalize(rb, 2)
        
        # we need to take the mean pooling taking into account the padding
        # tensors are of dim batch_size x samples x output_size x (len - kernel + 1)
        # pad tensors are of dim batch_size x samples x (len - kernel + 1)
        
        pid_title_pad_ex = torch.unsqueeze(pid_title_pad, 2).expand_as(pt)
        pid_body_pad_ex = torch.unsqueeze(pid_body_pad, 2).expand_as(pb)
        rest_title_pad_ex = torch.unsqueeze(rest_title_pad, 2).expand_as(rt)
        rest_body_pad_ex = torch.unsqueeze(rest_body_pad, 2).expand_as(rb)
        
        pt = torch.squeeze(torch.sum(pt * pid_title_pad_ex, dim = 3), dim = 3)
        pb = torch.squeeze(torch.sum(pb * pid_body_pad_ex, dim = 3), dim = 3)
        rt = torch.squeeze(torch.sum(rt * rest_title_pad_ex, dim = 3), dim = 3)
        rb = torch.squeeze(torch.sum(rb * rest_body_pad_ex, dim = 3), dim = 3)

        # tensors are not of dim batch_size x samples x output_size
        # need to scale down because not all uniformly padded
        
        ptp_norm = torch.sum(pid_title_pad, dim = 2).clamp(min = 1).expand_as(pt)
        pbp_norm = torch.sum(pid_body_pad, dim = 2).clamp(min = 1).expand_as(pb)
        rtp_norm = torch.sum(rest_title_pad, dim = 2).clamp(min = 1).expand_as(rt)
        rbp_norm = torch.sum(rest_body_pad, dim = 2).clamp(min = 1).expand_as(rb)
        
        pt = pt / ptp_norm
        pb = pb / pbp_norm
        rt = rt / rtp_norm
        rb = rb / rbp_norm
        
        pid_tensor = (pt + pb)/2
        rest_tensor = (rt + rb)/2
        #pid_tensor = pb
        #rest_tensor = rb
        
        if is_training:
            loss = loss_function(pid_tensor, rest_tensor, margin = 1.0)
            loss.backward()
            losses.append(loss.cpu().data[0])
            optimizer.step()
        else:
            expanded = pid_tensor.expand_as(rest_tensor)
            similarity = cs(expanded, rest_tensor, dim=2).squeeze(2)
            similarity = similarity.data.cpu().numpy()
            labels = batch['labels'].numpy()
            l = dr.convert(similarity, labels)
            losses.extend(l)

    # Calculate epoch level scores
    if is_training:
        avg_loss = np.mean(losses)
        return avg_loss
    else:
        e = Evaluation(losses)
        MAP = e.MAP()*100
        MRR = e.MRR()*100
        P1 = e.Precision(1)*100
        P5 = e.Precision(5)*100
        return (MAP, MRR, P1, P5)

In [28]:
model = CNN(700, 3, embedding_tensor, 0.2)

In [29]:
val_path = "../data/dev.txt"
val = dr.read_annotations(val_path, K_neg = -1, prune_pos_cnt = -1)
val_ex = dr.create_dev_set(ids_corpus, val)

In [30]:
z = train_model(train_ex, val_ex, model)

-------------
Epoch 1:

Train loss: 0.992593
Val MAP: 53.729437, MRR: 65.864563, P1: 51.851852, P5: 43.386243
This epoch took: 87.819034
-------------
Epoch 2:

Train loss: 0.863300
Val MAP: 54.558940, MRR: 66.492802, P1: 51.851852, P5: 44.550265
This epoch took: 87.993196
-------------
Epoch 3:

Train loss: 0.833603
Val MAP: 55.201853, MRR: 67.830126, P1: 53.968254, P5: 44.973545
This epoch took: 88.046032
-------------
Epoch 4:

Train loss: 0.818723
Val MAP: 55.792063, MRR: 68.290753, P1: 55.026455, P5: 45.185185
This epoch took: 87.912399
-------------
Epoch 5:

Train loss: 0.807664
Val MAP: 55.984899, MRR: 67.440613, P1: 53.439153, P5: 45.608466
This epoch took: 87.949647
-------------
Epoch 6:

Train loss: 0.799101
Val MAP: 56.268920, MRR: 68.281052, P1: 55.026455, P5: 45.079365
This epoch took: 87.850875
-------------
Epoch 7:

Train loss: 0.792169
Val MAP: 56.211172, MRR: 68.610540, P1: 56.084656, P5: 44.761905
This epoch took: 87.828775
-------------
Epoch 8:

Train loss: 0.786

In [38]:
model = torch.load("model4")
real_val_path = "../data/test.txt"
real_val = dr.read_annotations(real_val_path, K_neg = -1, prune_pos_cnt = -1)
real_val_ex = dr.create_dev_set(ids_corpus, real_val)
model = model.cuda()
z = run_epoch(real_val_ex, False, model, None)

In [39]:
z

(56.528459899168816, 68.96298464316293, 55.376344086021504, 43.655913978494645)